In [ ]:
# Final integration script (ORA) - notebook-friendly version
# Purpose:
# 1) Read ORA results for core TRS features, and metabolomics enrichment results
# 2) Standardize pathway names, find overlapping pathways
# 3) Save integrated table and generate a publication-ready scatter plot
#
# Usage in notebook: paste into a code cell and run. Edit IO_DIR below to point to your data folder.

In [ ]:
import os
from pathlib import Path
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ------------------------------------------------------------
# User-editable settings
# ------------------------------------------------------------
# By default use a 'multi_omics_results' folder under current working directory.
# You may replace IO_DIR with your absolute path, e.g. r"D:\your\path\multi_omics_results"
IO_DIR = os.path.join(os.getcwd(), "multi_omics_results")

# Input filenames (inside IO_DIR)
CORE_ORA_FILENAME = "ora_given_genes_kegg_results.csv"    # ORA results for core TRS features
METABOLITE_FILENAME = "dem_kegg_enrichment.xlsx"         # Metabolome enrichment results

# Output filenames (will be written into IO_DIR)
ORA_RESULTS_COPY = "ora_kegg_results.csv"                        # optional copy of ORA
FINAL_TABLE_FILENAME = "final_integrated_pathways_ORA.csv"       # integrated CSV
FINAL_PLOT_PNG = "final_integrated_pathways_plot_ORA.png"        # final PNG
FINAL_PLOT_PDF = "final_integrated_pathways_plot_ORA.pdf"        # optional PDF

# Optional parameters
CORE_GENE_COUNT = 50    # informational only
GENERATE_PDF = True     # set to False to skip PDF export

In [ ]:
# ------------------------------------------------------------
# Utility functions
# ------------------------------------------------------------
def safe_read_csv(path):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"CSV file not found: {path}")
    df = pd.read_csv(p)
    print(f"Read CSV: {p} ({len(df)} rows, {df.shape[1]} columns)")
    return df

def safe_read_excel(path):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Excel file not found: {path}")
    df = pd.read_excel(p, engine="openpyxl")
    print(f"Read Excel: {p} ({len(df)} rows, {df.shape[1]} columns)")
    return df

def standardize_pathway_name_series(s: pd.Series) -> pd.Series:
    # Convert to string, remove ' Homo sapiens' suffix and trailing parentheses content, strip whitespace
    s = s.fillna("").astype(str)
    s = s.str.split(" Homo sapiens").str[0].str.strip()
    s = s.str.replace(r"\s*\(.*\)$", "", regex=True).str.strip()
    return s

In [ ]:
# ------------------------------------------------------------
# Prepare paths
# ------------------------------------------------------------
io_dir = Path(IO_DIR)
if not io_dir.exists():
    print(f"IO_DIR does not exist: {io_dir!s}")
    print("Please create the folder or set IO_DIR to the directory that contains your input files.")
    # Do not exit automatically in notebook - raise to stop flow
    raise FileNotFoundError(f"IO_DIR not found: {io_dir}")

core_ora_path = io_dir / CORE_ORA_FILENAME
metabolite_path = io_dir / METABOLITE_FILENAME
ora_results_copy_path = io_dir / ORA_RESULTS_COPY
final_table_path = io_dir / FINAL_TABLE_FILENAME
final_plot_png_path = io_dir / FINAL_PLOT_PNG
final_plot_pdf_path = io_dir / FINAL_PLOT_PDF

print(f"IO directory: {io_dir}")
print(f"Core ORA file: {core_ora_path}")
print(f"Metabolite file: {metabolite_path}")

In [ ]:
# ------------------------------------------------------------
# Read input files
# ------------------------------------------------------------
try:
    core_ora_df = safe_read_csv(core_ora_path)
except Exception as e:
    print("Error reading core ORA file:", e)
    raise

try:
    metabolite_df = safe_read_excel(metabolite_path)
except Exception as e:
    print("Error reading metabolite file:", e)
    raise

In [ ]:
# ------------------------------------------------------------
# Standardize pathway names
# ------------------------------------------------------------
print("\n--- Standardizing pathway names ---")

# For core ORA
if "Term" in core_ora_df.columns:
    core_ora_df["Pathway_Name"] = standardize_pathway_name_series(core_ora_df["Term"])
else:
    # fallback: try to find a reasonable column
    candidates = [c for c in core_ora_df.columns if "term" in c.lower() or "path" in c.lower()]
    if candidates:
        core_ora_df["Pathway_Name"] = standardize_pathway_name_series(core_ora_df[candidates[0]])
        print(f"Using column '{candidates[0]}' as pathway name for core ORA.")
    else:
        core_ora_df["Pathway_Name"] = standardize_pathway_name_series(core_ora_df.iloc[:, 0])
        print("Fallback: using first column as pathway name for core ORA.")

# For metabolome
if "name" in metabolite_df.columns:
    metabolite_df["Pathway_Name"] = metabolite_df["name"].astype(str).str.strip()
else:
    candidates = [c for c in metabolite_df.columns if "name" in c.lower() or "path" in c.lower()]
    if candidates:
        metabolite_df["Pathway_Name"] = metabolite_df[candidates[0]].astype(str).str.strip()
        print(f"Using column '{candidates[0]}' as pathway name for metabolome.")
    else:
        metabolite_df["Pathway_Name"] = metabolite_df.iloc[:, 0].astype(str).str.strip()
        print("Fallback: using first column as pathway name for metabolome.")

print("Sample transcriptome pathway names:", core_ora_df["Pathway_Name"].head(5).tolist())
print("Sample metabolome pathway names:", metabolite_df["Pathway_Name"].head(5).tolist())

In [ ]:
# ------------------------------------------------------------
# Identify p-value / FDR columns
# ------------------------------------------------------------
print("\n--- Identifying significance columns ---")
# Transcriptome ORA p-value candidate
transcriptome_p_candidates = [c for c in core_ora_df.columns if re.search(r"p[_\-]?value|pval|p_val|^p$", c, flags=re.I)]
if not transcriptome_p_candidates:
    # broaden search
    transcriptome_p_candidates = [c for c in core_ora_df.columns if "p" in c.lower() and ("value" in c.lower() or "val" in c.lower())]
if transcriptome_p_candidates:
    p_col = transcriptome_p_candidates[0]
    core_ora_df["P_transcriptome"] = pd.to_numeric(core_ora_df[p_col], errors="coerce")
    print(f"Using '{p_col}' as transcriptome p-value column.")
else:
    core_ora_df["P_transcriptome"] = np.nan
    print("No transcriptome p-value column detected. 'P_transcriptome' set to NaN.")


In [ ]:
# Metabolome FDR candidate
met_fdr_candidates = [c for c in metabolite_df.columns if re.search(r"fdr|padj|adj|adjust", c, flags=re.I)]
if not met_fdr_candidates:
    met_fdr_candidates = [c for c in metabolite_df.columns if re.search(r"p[_\-]?value|pval|p_val|^p$", c, flags=re.I)]
if met_fdr_candidates:
    met_fdr_col = met_fdr_candidates[0]
    metabolite_df["FDR_metabolome"] = pd.to_numeric(metabolite_df[met_fdr_col], errors="coerce")
    print(f"Using '{met_fdr_col}' as metabolome FDR/p-value column.")
else:
    metabolite_df["FDR_metabolome"] = np.nan
    print("No metabolome FDR/p-value column detected. 'FDR_metabolome' set to NaN.")

In [ ]:
# Hits column (number of metabolites hitting the pathway)
if "Hits" in metabolite_df.columns:
    metabolite_df["Hits"] = pd.to_numeric(metabolite_df["Hits"], errors="coerce").fillna(0).astype(int)
else:
    hits_candidates = [c for c in metabolite_df.columns if re.search(r"hit|count|num", c, flags=re.I)]
    if hits_candidates:
        metabolite_df["Hits"] = pd.to_numeric(metabolite_df[hits_candidates[0]], errors="coerce").fillna(0).astype(int)
        print(f"Using '{hits_candidates[0]}' as hits column.")
    else:
        metabolite_df["Hits"] = 1  # default to 1 if no count available
        print("No hits/count column found in metabolome; defaulting 'Hits' to 1.")

In [ ]:
# ------------------------------------------------------------
# Merge tables by standardized pathway name
# ------------------------------------------------------------
print("\n--- Merging ORA and metabolome results by pathway name ---")
merged = pd.merge(
    core_ora_df,
    metabolite_df[["Pathway_Name", "FDR_metabolome", "Hits"]],
    on="Pathway_Name",
    how="inner"
)

if merged.empty:
    print("Warning: No overlapping pathways between transcriptome ORA and metabolome results.")
else:
    print(f"Found {len(merged)} overlapping pathways.")

# Compute -log10 values for plotting
merged["-log10(P_transcriptome)"] = merged["P_transcriptome"].apply(lambda x: -np.log10(x) if pd.notna(x) and x > 0 else np.nan)
merged["-log10(FDR_metabolome)"] = merged["FDR_metabolome"].apply(lambda x: -np.log10(x) if pd.notna(x) and x > 0 else np.nan)

# Reorder columns for output
out_cols = ["Pathway_Name", "P_transcriptome", "-log10(P_transcriptome)", "FDR_metabolome", "-log10(FDR_metabolome)", "Hits"]
for c in out_cols:
    if c not in merged.columns:
        merged[c] = np.nan
final_integrated_df = merged[out_cols]

In [ ]:
# ------------------------------------------------------------
# Save final integrated table
# ------------------------------------------------------------
try:
    final_integrated_df.to_csv(final_table_path, index=False)
    print(f"\nSaved final integrated table: {final_table_path}")
except Exception as e:
    print("Failed to write final integrated table:", e)

# Optional: save a copy of the ORA input (helps reproducibility)
try:
    core_ora_df.to_csv(ora_results_copy_path, index=False)
    print(f"Saved a copy of the core ORA table: {ora_results_copy_path}")
except Exception as e:
    print("Could not save ORA results copy:", e)

In [ ]:
# ------------------------------------------------------------
# Plot results (bubble scatter)
# ------------------------------------------------------------
def plot_integration(df, png_path=None, pdf_path=None):
    if df is None or df.empty:
        print("No data to plot.")
        return

    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(10, max(6, len(df) * 0.4)))

    sizes = (df["Hits"].fillna(1).astype(float) * 80).clip(lower=20)

    scatter = ax.scatter(
        x=df["-log10(P_transcriptome)"],
        y=df["-log10(FDR_metabolome)"],
        s=sizes,
        alpha=0.75,
        edgecolors="w",
        linewidth=0.8,
        c="steelblue",
    )
    # Add labels for points where both coordinates are present
    for _, row in df.iterrows():
        x = row["-log10(P_transcriptome)"]
        y = row["-log10(FDR_metabolome)"]
        if pd.notna(x) and pd.notna(y):
            ax.text(x, y + 0.05, str(row["Pathway_Name"]), fontsize=8, ha="center", va="bottom")

    ax.set_xlabel("-log10(P_transcriptome) from Core TRS Features ORA", fontweight="bold")
    ax.set_ylabel("-log10(FDR) from Metabolome Enrichment", fontweight="bold")
    ax.set_title("Integrated Pathway Analysis (Transcriptome ORA vs Metabolome)", fontsize=14, fontweight="bold")

    ax.axvline(x=-np.log10(0.05), color="red", linestyle="--", alpha=0.6, label="p=0.05")
    ax.axhline(y=-np.log10(0.05), color="red", linestyle="--", alpha=0.6, label="FDR=0.05")

    # Legend for hit sizes (show a few typical sizes)
    unique_hits = sorted(df["Hits"].dropna().unique())
    # limit to up to 4 legend markers
    legend_hits = unique_hits[:4]
    for hits in legend_hits:
        ax.scatter([], [], s=max(20, hits * 80), c="steelblue", alpha=0.7, edgecolors="w", linewidth=0.8, label=f"{hits} metabolites")
    ax.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc="upper left")

    plt.tight_layout()
    if png_path:
        plt.savefig(png_path, dpi=300, bbox_inches="tight")
        print(f"Saved plot PNG: {png_path}")
    if pdf_path:
        plt.savefig(pdf_path, dpi=300, bbox_inches="tight")
        print(f"Saved plot PDF: {pdf_path}")
    plt.show()
    plt.close()

# Generate plot
plot_integration(final_integrated_df, png_path=final_plot_png_path, pdf_path=final_plot_pdf_path if GENERATE_PDF else None)

print("\n--- Integration pipeline completed ---")